In [1]:
!pip install torch torchvision


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
!pip install scikit-learn


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install tqdm


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install pandas


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import torch
import torch.nn as nn
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms.functional import to_pil_image
from PIL import Image
import os
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


# CHANGE THIS

In [3]:
##MAKE SURE TO CHANGE THIS
best_model_path = "amb_models/starting_lo_model.pth"
target_col = 'Lung Opacity'

In [4]:
class CSVDataset(Dataset):
    def __init__(self, dataframe, image_root_dir, target_columns=None, transform=None,
                 save_dir=None, use_saved_images=False):
        self.data = dataframe
        self.image_root_dir = image_root_dir
        self.target_columns = target_columns
        self.transform = transform
        self.save_dir = save_dir
        self.use_saved_images = use_saved_images

        if self.save_dir:
            os.makedirs(self.save_dir, exist_ok=True)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        
        # Use index for the saved tensor filename
        image_index = row['Unnamed: 0']
        saved_image_path = os.path.join(self.save_dir, f"{image_index}.pt")

        if self.use_saved_images:
            if os.path.exists(saved_image_path):
                image_tensor = torch.load(saved_image_path)
            else:
                raise FileNotFoundError(f"Saved tensor not found: {saved_image_path}")
        else:
            original_image_path = os.path.join(self.image_root_dir, row['Path'])
            image = Image.open(original_image_path).convert("L")
            image_tensor = self.transform(image) if self.transform else transforms.ToTensor()(image)

            if self.save_dir:
                torch.save(image_tensor, saved_image_path)

        if self.target_columns:
            labels = pd.to_numeric(row[self.target_columns], errors='coerce').fillna(0).astype(float).values
            labels = torch.tensor(labels, dtype=torch.float32)
            return image_tensor, labels

        return image_tensor

In [5]:
class MultiLabelResNet50(nn.Module):
    def __init__(self, num_classes):
        super(MultiLabelResNet50, self).__init__()
        
        # Load pre-trained ResNet50
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        
        # Modify the fully connected layer for multi-label classification
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, 512),  # New intermediate layer
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout to prevent overfitting
            nn.Linear(512, num_classes),  # Output layer
            nn.Sigmoid()  # Sigmoid for multi-label classification (soften the data)
            #nn.Tanh()  #This is between -1 and 1

           # nn.Linear(self.base_model.fc.in_features, num_classes),
           # nn.Sigmoid()  # Sigmoid activation for multi-label classification
        )

    def forward(self, x):
        return self.base_model(x)

In [13]:
image_root = '/central/groups/CS156b/2025/CodeMonkeys/input_images'
image_root_dir = "input_images/train_contrast"
train_save_dir = os.path.join(image_root, 'train')

def get_filtered_df(col, num=None):
    
    #test_save_dir = os.path.join(image_root, 'test')
    full_train_df = pd.read_csv('train2023.csv')
    #filtered_train_df = full_train_df.iloc[:29692]
    if num != None:
        full_train_df = full_train_df.iloc[:num]

    filtered_train_df = full_train_df.dropna(subset=[col]).copy()
    filtered_train_df[col] = (filtered_train_df[col] + 1) / 2
    #filtered_train_df = filtered_train_df[filtered_train_df[col] != 0.5]  # Drop rows with 'Pleural_Effusion' == 0
    return filtered_train_df

filtered_train_df = get_filtered_df(target_col)

In [14]:
label_counts = filtered_train_df[target_col].value_counts()
print(label_counts)


Lung Opacity
1.0    84352
0.5     5310
0.0     5102
Name: count, dtype: int64


In [15]:
from sklearn.model_selection import train_test_split

# Define your target columns once
target_columns = [target_col]

# Step 1: Split the dataframe
train_df, val_df = train_test_split(filtered_train_df, test_size=0.15, random_state=42)

# Step 2: Create training dataset
train_dataset = CSVDataset(
    dataframe=train_df, 
    image_root_dir=image_root, 
    target_columns=target_columns, 
    save_dir=train_save_dir, 
    use_saved_images=True
)

# Step 3: Create validation dataset
val_dataset = CSVDataset(
    dataframe=val_df, 
    image_root_dir=image_root, 
    target_columns=target_columns, 
    save_dir=train_save_dir, 
    use_saved_images=True
)

# Step 4: Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [17]:
lo_labels = train_df[target_col].values
label_map = {0.0: 0, 0.5: 1, 1.0: 2}
mapped_labels = np.array([label_map[float(lbl)] for lbl in lo_labels])

class_counts = np.bincount(mapped_labels)
weights = 1. / (class_counts + 1e-6)
sample_weights = torch.tensor(weights[mapped_labels], dtype=torch.float)

sampler = torch.utils.data.WeightedRandomSampler(sample_weights, len(sample_weights), replacement=True)
train_loader = DataLoader(train_dataset, batch_size=16, sampler=sampler)

In [19]:
def freeze_base_layers(model, until_layer=6):
    """
    Freeze layers of ResNet-50 up to a certain stage (e.g., until_layer=6 means keep layers 0-5 frozen).
    """
    child_counter = 0
    for child in model.base_model.children():
        if child_counter < until_layer:
            for param in child.parameters():
                param.requires_grad = False
        child_counter += 1
    return model


In [21]:
uncertain_weight_factor = 0.25
class_weights = {}

# Loop over each target column
for col in target_columns:
    # Count the occurrences of each class in the column
    counts = filtered_train_df[col].value_counts()
    total = len(filtered_train_df[col])
    
    # Calculate class weights using inverse frequency (you can also experiment with other strategies)
    weights = {
        0: total / (counts.get(0, 0) + 1),  # Add 1 to avoid division by zero
        0.5: total / (counts.get(0.5, 0) + 1) * uncertain_weight_factor,
        1: total / (counts.get(1, 0) + 1)
    }
    
    # Store weights for each class
    class_weights[col] = weights
class_weights[target_col] = {0: 1.0, 0.5: 0.25, 1: 1.0}  ##GET RID OF THIS LINE IF DONT HAVE SAMPLER
# Example: Print out the weights for each class
for col in target_columns:
    print(f"Class weights for {col}: {class_weights[col]}")


Class weights for Lung Opacity: {0: 1.0, 0.5: 0.25, 1: 1.0}


In [22]:
criterion = nn.MSELoss(reduction='none')

def masked_MSE_loss(output, target, class_weights):
    # Create a mask for non-NaN target values
    mask = ~torch.isnan(target)
    
    # Apply the MSE loss
    loss = criterion(output, target)
    
    # Loop through each class and apply the class weights
    for class_idx, col in enumerate(target_columns):
        # Get the class values for the current class
        class_values = target[:, class_idx]
        
        # Apply the class weights to each class value
        weight = torch.tensor([class_weights[col].get(x.item(), 1) for x in class_values], dtype=torch.float32, device=output.device)
        
        # Apply the weight to the loss (broadcast the weight to match the loss shape)
        loss = loss * mask  # Apply mask to exclude NaN targets
        loss[:, class_idx] *= weight  # Apply weight per class
    
    # Return mean loss for valid entries
    return loss.sum() / mask.sum()


## No Grid Search

In [ ]:
import numpy as np
import torch
import torch.optim as optim
import os

# Hyperparameters and model setup
num_classes = 1  # Predicting 'Pleural Effusion'
model = MultiLabelResNet50(num_classes=num_classes).to(device)
model = freeze_base_layers(model, until_layer=4)  # Freeze layers
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

# Early stopping parameters
early_stopping_patience = 3
best_val_loss = float('inf')
patience_counter = 0
os.makedirs("models", exist_ok=True)

num_epochs = 30
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = masked_MSE_loss(outputs, labels, class_weights)
        running_loss += loss.item()

        loss.backward()
        optimizer.step()

        predicted_class = torch.where(outputs > 0.5, torch.tensor(1.0).to(device),
                                      torch.where(outputs < -0.5, torch.tensor(-1.0).to(device), torch.tensor(0.0).to(device)))
        correct += (predicted_class == labels).sum().item()
        total += labels.numel()

    avg_loss = running_loss / len(train_loader)
    accuracy = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = masked_MSE_loss(outputs, labels, class_weights)
            val_loss += loss.item()

            predicted_class = torch.where(
                outputs < 0.25, torch.tensor(0.0).to(device),
                torch.where(
                    outputs < 0.75, torch.tensor(0.5).to(device),
                    torch.tensor(1.0).to(device)
                )
            )

            all_preds.append(predicted_class.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

            val_correct += (predicted_class == labels).sum().item()
            val_total += labels.numel()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = val_correct / val_total

    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Early stopping check
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        patience_counter = 0
        torch.save(model.state_dict(), best_model_path)
        #print("✅ Saved new best model.")
    else:
        patience_counter += 1
     #   print(f"⏳ No improvement. Patience: {patience_counter}/{early_stopping_patience}")

    # Stop if patience exceeded
    if patience_counter >= early_stopping_patience:
        print("⛔ Early stopping triggered.")
        break

    #Optional: save every epoch
    #if (epoch + 1) % 10 == 0:
    torch.save(model.state_dict(), f"amb_models/loc_epoch_{epoch+1}.pth")


## With Grid Search

In [ ]:
import itertools
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models
from sklearn.metrics import precision_score, recall_score, f1_score

# Hyperparameter grid
hidden_sizes = [256, 512]
frozen_layers = [3, 5]
learning_rates = [5e-4, 1e-4]

# Results dictionary
results = []

# Model definition
class MultiLabelResNet50(nn.Module):
    def __init__(self, num_classes, hidden_size):
        super(MultiLabelResNet50, self).__init__()
        self.base_model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        self.base_model.fc = nn.Sequential(
            nn.Linear(self.base_model.fc.in_features, hidden_size),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(hidden_size, num_classes),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.base_model(x)

# Freeze function
def freeze_base_layers(model, until_layer):
    child_counter = 0
    for child in model.base_model.children():
        if child_counter < until_layer:
            for param in child.parameters():
                param.requires_grad = False
        child_counter += 1
    return model

# Grid search loop
for hidden_size, freeze_until, lr in itertools.product(hidden_sizes, frozen_layers, learning_rates):
    print(f"Training model with hidden_size={hidden_size}, freeze_until={freeze_until}, lr={lr}")

    # Initialize model and optimizer
    model = MultiLabelResNet50(num_classes=1, hidden_size=hidden_size).to(device)
    model = freeze_base_layers(model, until_layer=freeze_until)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Training
    for epoch in range(3):
        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = masked_MSE_loss(outputs, labels, class_weights)
            loss.backward()
            optimizer.step()

    # Validation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = masked_MSE_loss(outputs, labels, class_weights)
            val_loss += loss.item()

            # Custom thresholding
            predicted = torch.where(
                outputs < 0.25, torch.tensor(0.0).to(device),
                torch.where(
                    outputs < 0.75, torch.tensor(0.5).to(device),
                    torch.tensor(1.0).to(device)
                )
            )

            all_preds.append(predicted.cpu().numpy())
            all_labels.append(labels.cpu().numpy())
            val_correct += (predicted == labels).sum().item()
            val_total += labels.numel()

    # Compute metrics
    all_preds = np.concatenate(all_preds)
    all_labels = np.concatenate(all_labels)
    
    # Map 0.0 → 0, 0.5 → 1, 1.0 → 2
    all_preds = (all_preds * 2).astype(int)
    all_labels = (all_labels * 2).astype(int)

    val_accuracy = val_correct / val_total
    val_precision = precision_score(all_labels, all_preds, average='macro', zero_division=1)
    val_recall = recall_score(all_labels, all_preds, average='macro', zero_division=1)
    val_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=1)
    avg_val_loss = val_loss / len(val_loader)

    # Store results
    results.append({
        'hidden_size': hidden_size,
        'freeze_until': freeze_until,
        'learning_rate': lr,
        'val_loss': avg_val_loss,
        'val_accuracy': val_accuracy,
        'val_precision': val_precision,
        'val_recall': val_recall,
        'val_f1': val_f1
    })

# Display sorted configs
results.sort(key=lambda x: x['val_f1'], reverse=True)
for r in results:
    print(r)


Training model with hidden_size=256, freeze_until=3, lr=0.0005
Training model with hidden_size=256, freeze_until=3, lr=0.0001
Training model with hidden_size=256, freeze_until=5, lr=0.0005
Training model with hidden_size=256, freeze_until=5, lr=0.0001
Training model with hidden_size=512, freeze_until=3, lr=0.0005
Training model with hidden_size=512, freeze_until=3, lr=0.0001
Training model with hidden_size=512, freeze_until=5, lr=0.0005


## Testing Model on Validation Set

In [19]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np
import torch

def evaluate_model_per_class(val_loader, device, model_path):
    model = MultiLabelResNet50(num_classes=1).to(device)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    all_preds = []
    all_labels = []
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = masked_MSE_loss(outputs, labels, class_weights)
            val_loss += loss.item()

            predicted_class = torch.where(outputs > 0, torch.tensor(1.0).to(device), torch.tensor(-1.0).to(device))

            all_preds.append(predicted_class.cpu().numpy())
            all_labels.append(labels.cpu().numpy())

            val_correct += (predicted_class == labels).sum().item()
            val_total += labels.numel()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = val_correct / val_total

    # Flatten for sklearn metrics
    all_preds = np.concatenate(all_preds, axis=0)
    all_labels = np.concatenate(all_labels, axis=0)

    val_precision = precision_score(all_labels, all_preds, average='macro', zero_division=1)
    val_recall = recall_score(all_labels, all_preds, average='macro', zero_division=1)
    val_f1 = f1_score(all_labels, all_preds, average='macro', zero_division=1)

    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    print(f"Validation Precision: {val_precision:.4f}, Validation Recall: {val_recall:.4f}, Validation F1: {val_f1:.4f}")


# Example usage
evaluate_model_per_class(val_loader, device, best_model_path)


ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [ ]:
import torch
from collections import Counter
import numpy as np

def print_label_distribution(val_loader):
    all_labels = []

    for _, labels in val_loader:
        all_labels.append(labels)

    all_labels = torch.cat(all_labels, dim=0).cpu().numpy()  # shape: (num_samples, num_classes)

    num_classes = all_labels.shape[1]

    print("Label distribution per class:")
    for i in range(num_classes):
        unique, counts = np.unique(all_labels[:, i], return_counts=True)
        dist = dict(zip(unique, counts))
        print(f"Class {i}: {dist}")
print_label_distribution(val_loader)

Label distribution per class:
Class 0: {np.float32(-1.0): np.int64(1829), np.float32(0.0): np.int64(169), np.float32(1.0): np.int64(252)}
Class 1: {np.float32(-1.0): np.int64(258), np.float32(0.0): np.int64(1891), np.float32(1.0): np.int64(101)}
Class 2: {np.float32(-1.0): np.int64(192), np.float32(0.0): np.int64(1811), np.float32(1.0): np.int64(247)}
Class 3: {np.float32(-1.0): np.int64(56), np.float32(0.0): np.int64(1188), np.float32(1.0): np.int64(1006)}
Class 4: {np.float32(-1.0): np.int64(34), np.float32(0.0): np.int64(2190), np.float32(1.0): np.int64(26)}
Class 5: {np.float32(-1.0): np.int64(409), np.float32(0.0): np.int64(1045), np.float32(1.0): np.int64(796)}
Class 6: {np.float32(-1.0): np.int64(3), np.float32(0.0): np.int64(2199), np.float32(1.0): np.int64(48)}
Class 7: {np.float32(-1.0): np.int64(47), np.float32(0.0): np.int64(2113), np.float32(1.0): np.int64(90)}
Class 8: {np.float32(-1.0): np.int64(50), np.float32(0.0): np.int64(1164), np.float32(1.0): np.int64(1036)}


In [ ]:
# Create dataset
test_dataset = CSVDataset(
    dataframe=df_first10rows_test, 
    image_root_dir=image_root, 
    target_columns=None, 
    transform=image_transforms,  # Pass the transform
    save_dir=test_save_dir, 
    use_saved_images=False  # Set to True if you want to load tensors from CSV
)

# Create DataLoader
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Iterate through batches
for batch_idx, (images) in enumerate(test_loader):
    print(f"Batch {batch_idx + 1}")
    print("Images shape:", images.shape)

## Run Model

In [14]:
model_pe = MultiLabelResNet50(num_classes=1).to(device)
model_pe.load_state_dict(torch.load('models/best_pe1_model.pth'))
model_pe.eval()

model_cm = MultiLabelResNet50(num_classes=1).to(device)
model_cm.load_state_dict(torch.load('models/best_cm0_model.pth'))
model_cm.eval()

model_lo = MultiLabelResNet50(num_classes=1).to(device)
model_lo.load_state_dict(torch.load('amb_models/starting_lo_model.pth'))
model_lo.eval()

MultiLabelResNet50(
  (base_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [15]:
average_values = {
    "No Finding": -0.734655,
    "Enlarged Cardiomediastinum": -0.275805,
    "Cardiomegaly": 0.190770,
    "Lung Opacity": 0.836288,
    "Pneumonia": 0.031183,
    "Pleural Effusion": 0.384547,
    "Pleural Other": 0.521795,
    "Fracture": 0.392374,
    "Support Devices": 0.888289
}

In [17]:
import torch
import os
import pandas as pd
from tqdm import tqdm

# Directory containing the test images
test_dir = 'input_images/test'

# Columns for the prediction output
columns = ["Id", "No Finding", "Enlarged Cardiomediastinum", "Cardiomegaly", "Lung Opacity", 
           "Pneumonia", "Pleural Effusion", "Pleural Other", "Fracture", "Support Devices"]

# Average values for columns other than Pleural Effusion
average_values = {
    "No Finding": -0.734655,
    "Enlarged Cardiomediastinum": -0.275805,
    "Cardiomegaly": 0.190770,
    "Lung Opacity": 0.836288,
    "Pneumonia": 0.031183,
    "Pleural Other": 0.521795,
    "Fracture": 0.392374,
    "Support Devices": 0.888289
}

# Batch size for processing
batch_size = 64
batch = []
batch_filenames = []
predictions = []

file_list = [f for f in os.listdir(test_dir) if f.endswith(".pt")]

for filename in tqdm(file_list):
    image_path = os.path.join(test_dir, filename)
    image_tensor = torch.load(image_path).to(device)
    batch.append(image_tensor)
    batch_filenames.append(filename.split('.')[0])

    if len(batch) == batch_size or filename == file_list[-1]:
        input_batch = torch.stack(batch)

        with torch.no_grad():
            output_pe = model_pe(input_batch).cpu().numpy()
            output_cm = model_cm(input_batch).cpu().numpy()
            output_lo = model_lo(input_batch).cpu().numpy()

        for i in range(len(batch)):
            pe_score = output_pe[i][0] * 2 - 1  # Rescale from [0,1] to [-1,1] if needed
            cm_score = output_cm[i][0] * 2 - 1
            lo_score = output_lo[i][0] * 2 - 1

            row = [batch_filenames[i]]
            for col in columns[1:]:
                if col == "Pleural Effusion":
                    row.append(pe_score)
                elif col == "Cardiomegaly":
                    row.append(cm_score)
                elif col == "Lung Opacity":
                    row.append(lo_score)
                else:
                    row.append(average_values.get(col, 0))

            predictions.append(row)

        batch = []
        batch_filenames = []

# Save predictions
df_predictions = pd.DataFrame(predictions, columns=columns)
df_predictions = df_predictions.sort_values(by="Id")
df_predictions.to_csv('amb_test_predictions.csv', index=False)

print("Predictions saved to 'amb_test_predictions.csv'")


  0%|          | 0/22596 [00:00<?, ?it/s]

100%|██████████| 22596/22596 [10:37<00:00, 35.47it/s]


Predictions saved to 'amb_test_predictions.csv'


In [18]:
df_predictions = pd.DataFrame(predictions, columns=columns)
df_predictions = df_predictions.sort_values(by="Id", ascending=True)
df_predictions.head()

,Id,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Pneumonia,Pleural Effusion,Pleural Other,Fracture,Support Devices
963,100018,-0.734655,-0.275805,0.209412,-0.546575,0.031183,-0.925029,0.521795,0.392374,0.888289
14253,100019,-0.734655,-0.275805,-0.716572,-0.580686,0.031183,0.126851,0.521795,0.392374,0.888289
17256,100022,-0.734655,-0.275805,-0.716739,0.914831,0.031183,0.740513,0.521795,0.392374,0.888289
15017,100023,-0.734655,-0.275805,-0.613445,0.801272,0.031183,0.949374,0.521795,0.392374,0.888289
7126,100053,-0.734655,-0.275805,0.718571,-0.337341,0.031183,-0.666675,0.521795,0.392374,0.888289


In [ ]:
import os
print(os.getcwd())


/central/groups/CS156b/2025/CodeMonkeys
